# Welcome to ProgPy's Sim Value Example!

In this example notebook, we will demonstrate how to use ProgPy's [Simulation](https://nasa.github.io/progpy/prog_models_guide.html#simulation) methods to simulate a pneumatic value until threshold is met!

### Importing Modules

We'll be using ProgPy's [PneumaticValue](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#pneumatic-valve) Model for this example!

In ProgPy, the `PneumaticValveBase` class represents a prognostics model for a pneumatic valve. Pneumatic valves are devices that control the flow of compressed air or other gases in a pneumatic system. These systems are often used in industrial machinery and equipment.

This particular model captures the behavior of a pneumatic valve as it degrades over time due to wear and tear, and eventually fails. The model tracks the state of the valve through several state variables:

- `Aeb`, `Aet`, `Ai`: These represent the areas of leaks at the bottom and top pneumatic ports and internally in the valve, respectively.
- `k`: This is the spring coefficient, which decreases as the spring in the valve weakens over time.
- `mBot`, `mTop`: These represent the masses on the bottom and top of the piston, respectively.
- `r`: This is the friction coefficient, which increases as friction along the piston increases with wear.
- `v`: This is the velocity of the piston.
- `x`: This is the position of the piston.
- `pDiff`: This is the difference in pressure between the left and right sides of the valve.


In [ ]:
from progpy.models.pneumatic_valve import PneumaticValve

# Create a model object
valv = PneumaticValve(process_noise= 0)

Next, we'll need to define the [Future Load](https://nasa.github.io/progpy/prog_models_guide.html#future-loading) function for our Simulation method.

In [ ]:
cycle_time = 20
def future_loading(t, x=None):
        t = t % cycle_time
        if t < cycle_time/2:
            return valv.InputContainer({
                'pL': 3.5e5,
                'pR': 2.0e5,
                # Open Valve
                'uTop': False,
                'uBot': True
            })
        return valv.InputContainer({
            'pL': 3.5e5,
            'pR': 2.0e5,
            # Close Valve
            'uTop': True,
            'uBot': False
        })

In the `PneumaticValveBase` class of ProgPy, there are four key inputs that drive the behavior of the pneumatic valve model. These inputs represent the real-world conditions under which the valve operates. Let's discuss each of these inputs in detail:

- `pL`: This input represents the fluid pressure at the left side of the plug, measured in Pascals (Pa). It reflects the pressure of the fluid entering the valve from the left side.

- `pR`: Similarly, this input represents the fluid pressure at the right side of the plug, also measured in Pascals (Pa). It reflects the pressure of the fluid entering the valve from the right side.

- `uBot`: This is the input pressure at the bottom pneumatic port, measured in Pascals (Pa). This input pressure affects how the valve operates, including the opening and closing of the valve.

- `uTop`: This is the input pressure at the top pneumatic port, measured in Pascals (Pa). Similar to `uBot`, this input pressure also influences the operation of the valve.

In addition to these inputs, the model also takes several keyword arguments that represent various physical properties and parameters of the pneumatic valve, such as acceleration due to gravity (`g`), atmospheric pressure (`pAtm`), plug mass (`m`), spring offset distance (`offsetX`), stroke length (`Ls`), and surface area of the piston for gas contact (`Ap`). These parameters are used in the model's equations to simulate the physical behavior of the pneumatic valve.


Before we can simulate our model, we'll need to set our parameter for spring to 1 and define the first measured output of the PneumaticValue model. 
Furthermore, we need to define the first measured output. This is needed by the `simulate_to_threshold` method to initialize state!

In [ ]:
# Set wear parameter for spring to 1
valv.parameters['x0']['wk'] = 1

# Define first measured output.
first_output = valv.output(valv.initialize(future_loading(0)))

In the context of our `PneumaticValve` model, "wear" refers to the deterioration or damage that accumulates over time due to the operation of the pneumatic valve. This wear can impact various components of the valve, leading to changes in its performance or even failure.

Our model includes several wear parameters, each representing the rate of wear or degradation for different components of the valve. Let me break it down for you:

- `wb`: This is the wear parameter for the bottom leak. <span style="color:teal">It represents the rate at which the area of the leak at the bottom pneumatic port (`Aeb`) expands over time.</span>

- `wi`: This is the wear parameter for the internal leak. <span style="color:teal">It illustrates how quickly the area of the internal leak (`Ai`) increases as time passes.</span>

- `wt`: This is the wear parameter for the top leak. <span style="color:teal">It shows the rate at which the area of the leak at the top pneumatic port (`Aet`) enlarges over time.</span>

- `wk`: This wear parameter is for the spring. <span style="color:teal">It signifies how the spring coefficient (`k`) decreases over time due to wear and tear.</span>

- `wr`: This is the wear parameter for friction. <span style="color:teal">It represents the rate at which the friction coefficient (`r`) increases over time due to wear.</span>


With our future loading function defined, we can now simulate and plot the results of our model!

In [ ]:
# Simulate to threshold
print('\n\n------------------------------------------------')
print('Simulating to threshold\n\n')
# Configure options
config = {
    'dt': 0.1,
    'horizon': 800,
    'save_freq': 60,
    # 'print': True,
    # 'progress': True,
}
# Simulate
simulated_results = valv.simulate_to_threshold(future_loading, first_output, **config)
simulated_results.outputs.plot(compact = False, title = 'Outputs', xlabel = 'time', ylabel = '') # Plots may show up twice. Product of Jupyter Notebook.

Let's take a closer look at what the output of the simulated results are.

The `PneumaticValveBase` class in ProgPy provides us with several outputs that give us insights into different aspects of the valve's behavior. These outputs are:

- `Q`: This is the flow rate of the air or gas through the valve. It provides us with information about how much fluid is passing through the valve per unit time.

- `iB`: This is a boolean value that lets us know whether the piston of the valve is at the bottom. Depending on the design of the valve, this could be used to indicate whether the valve is in the closed position.

- `iT`: Similar to `iB`, this is a boolean value that signifies whether the piston of the valve is at the top. In some valve designs, this could be used to indicate whether the valve is in the open position.

- `pB`: This output represents the pressure at the bottom of the valve. In practical applications, this could be measured using a pressure sensor.

- `pT`: This is the counterpart to `pB`, representing the pressure at the top of the valve. This could also be measured using a pressure sensor.

- `x`: This output gives us the position of the piston in the valve. Depending on the available instrumentation, this could be measured directly using a position sensor or inferred from other measurements.

Now, let's simulate to threshold once more, but this time, with a different wear mode. We'll reset the wear parameter for the spring to 0 and set the wear parameter for friction to 1.

Furthermore, we'll need to redefine the first measured value and pass that value to our simulate method to initialize state.

In [ ]:
valv.parameters['x0']['wk'] = 0
valv.parameters['x0']['wr'] = 1

# Define first measured output.
first_output = valv.output(valv.initialize(future_loading(0)))

Finally, let's simulate and plot the results! We will use the same configuration settings as before.

In [ ]:
print('\n\n------------------------------------------------')
print('Simulating to threshold\n\n')

simulated_results = valv.simulate_to_threshold(future_loading, first_output, **config)
simulated_results.outputs.plot(compact = False, title = 'Outputs', xlabel = 'time', ylabel = '', tight_layout=True) # Plots may show up twice. Product of Jupyter Notebook.

To conclude, the `PneumaticValue` model in ProgPy is a great tool for simulating the behavior of a pneumatic valve as it degrades over time. It can be used to gain insights into the performance of the valve and predict when it will fail.

For more information, please refer to our [Pneumatic Value](https://nasa.github.io/progpy/api_ref/prog_models/IncludedModels.html#pneumatic-valve) documentation. For more information on ProgPy, please refer to the [ProgPy Documentation](https://nasa.github.io/progpy/index.html)!